## HamLib Simulation Observable Benchmark Tests - WIP

This notebook is being used to flesh out the details of integrating the HamLib utility module, the HamLib kernel function, and the newly developed functions for generating groups of commuting terms.

From any specific Hamiltonian, we create a base evolution circuit with 0 - N Trotter steps, initialize the circuit, and append the necessary basis rotations before measuring and computing the values of energy and other related observables.

At a later point, the execution code and plotting code will also be integrated into the benchmark framework. For now, however, the focus is on developing the code necessary to evaluate the performance of observable execution using various optimization options.


### Setup Dependencies

This cell is used to pull in dependencies from the benchmark framework, the HamLib utility module, and the newly created observables module (in WIP).

In [1]:
from qiskit.quantum_info import Pauli, SparsePauliOp
import numpy as np
from math import sin, cos, pi
import time

import sys
sys.path.insert(1, "WIP_benchmarks")

# Observable Helper Functions
from observables import *

sys.path[1:1] = ["../_common"]

import hamlib_simulation_kernel
from hamlib_simulation_kernel import HamiltonianSimulation, kernel_draw, get_valid_qubits
from hamlib_simulation_kernel import initial_state, create_circuit   # would like to remove these
from hamlib_utils import create_full_filenames, construct_dataset_name
from hamiltonian_simulation_exact import HamiltonianSimulationExact, HamiltonianSimulation_Noiseless

sys.path[1:1] = ["../../_common", "../../_common/qiskit"]

import execute

execute.verbose = False
execute.verbose_time = False


### Load and Initialize HamLib Modules

Here we use the HamLib module to select a Hamiltonian from HamLib and initialize its options.  The API for these functions needs improvement.  The code here is taken from the current HamLib benchmark modules.

In [2]:
import hamlib_simulation_benchmark, hamlib_simulation_kernel, hamlib_utils

hamlib_simulation_benchmark.verbose = False
hamlib_simulation_kernel.verbose = False
hamlib_utils.verbose = False

hamlib_simulation_kernel.global_U = None
hamlib_simulation_kernel.global_enc = None
hamlib_simulation_kernel.global_ratio = None
hamlib_simulation_kernel.global_rinst = None
hamlib_simulation_kernel.global_h = None
hamlib_simulation_kernel.global_pbc_val = None

# get key infomation about the selected Hamiltonian
# DEVNOTE: Error handling here can be improved by simply returning False or raising exception
def get_hamiltonian_info(hamiltonian_name=None, init_state=None, K=None, t=None):
    try:
        hamlib_simulation_kernel.filename = create_full_filenames(hamiltonian_name)
        hamlib_simulation_kernel.dataset_name_template = construct_dataset_name(hamlib_simulation_kernel.filename)
    except ValueError:
        print(f"ERROR: cannot load HamLib data for Hamiltonian: {hamiltonian_name}")
        return
    
    if hamlib_simulation_kernel.dataset_name_template == "File key not found in data":
        print(f"ERROR: cannot load HamLib data for Hamiltonian: {hamiltonian_name}")
        return
    
    # Set default parameter values for the hamiltonians
    hamlib_simulation_kernel.set_default_parameter_values(hamlib_simulation_kernel.filename)
        
    # assume default init_state if not given
    if init_state == None:
        init_state = "checkerboard"


### Specify a Hamiltonian for Testing
Choose a Hamiltonian from HamLib or create a custom Hamiltonian


In [3]:
# create the HamLib Simulation kernel, and the associated Hamiltonian operator

# Qubit width of the Hamiltonian
num_qubits = 6

# Specify the desired Hamiltonian from HamLib 
# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'
# If no name, it means we use an explicitly specified Hamiltonian
hamiltonian_name = 'TFIM'

########### HamLib Hamiltonians

if hamiltonian_name == 'TFIM':
    hamlib_simulation_kernel.global_h = 2
    hamlib_simulation_kernel.global_pbc_val = 'pbc'

if hamiltonian_name == 'Fermi-Hubbard-1D':
    hamlib_simulation_kernel.global_h = 2
    hamlib_simulation_kernel.global_pbc_val = 'pbc'

if hamiltonian_name == 'Bose-Hubbard-1D':
    hamlib_simulation_kernel.global_pbc_val = 'nonpbc'
    hamlib_simulation_kernel.global_U = 10
    hamlib_simulation_kernel.global_enc = 'gray'
    
########### Explicit Hamiltonians

# classical simple Ising is ZZ
# TFIM ZZ + X  is transverse field
# + longitudinal field -> ZZ, X, and Z
def get_ising_hamiltonian(L, J, h, alpha=0):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * cos(alpha)) for i in range(0, L)]

    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()

# create explicit Hamiltonian
if hamiltonian_name == '':
    H = get_ising_hamiltonian(L=num_qubits, J=0.2, h=1.2, alpha=pi / 8)
    H_terms = H.to_list()
    
    # hamiltonian = H_terms
    # ham_terms = hamiltonian
    ham_terms = H_terms
    qc = None


### Create a Hamiltonian Simulation Circuit
Initialize the circuit to a known initial state, append 'K' Trotter steps for total time 't', followed by basis rotation gates.

In [4]:
# create the HamLib Simulation kernel, from the current Hamiltonian.
# We use a different method for kernel creation with HamLib than with custom Hamiltonians

# Parameters of Tortterized simulation
K = 1
t = 0.05

########### create Trotterized evolution circuit for HamLib Hamiltonians

# create Hamiltonian evolution circuit from HamLib
if hamiltonian_name != '':
    init_state = "checkerboard"

    # this function reads the HamLib file content for the specified Hamiltonian
    get_hamiltonian_info(hamiltonian_name=hamiltonian_name, init_state=init_state)
            
    qc, bitstring, ham_op = HamiltonianSimulation(
        num_qubits, 
        K=K, t=t,
        hamiltonian = hamiltonian_name, 
        init_state = init_state,
        method = 1, 
    )

    # convert SparsePauliOp to list form
    ham_terms = ham_op.to_list()
    
########### create Trotterized evolution circuit for custom Hamiltonian
else:
    pass

print(f"... Hamiltonian name = {hamiltonian_name}")
print(ham_terms)
print("")

print(f"... Trotterized Circuit, K={K}, t={t}")
if num_qubits < 10:
    print(qc)
    print("")


... Hamiltonian name = TFIM
[('ZZIIII', (1+0j)), ('ZIIIIZ', (1+0j)), ('IZZIII', (1+0j)), ('IIIIZZ', (1+0j)), ('IIZZII', (1+0j)), ('IIIZZI', (1+0j)), ('XIIIII', (2+0j)), ('IXIIII', (2+0j)), ('IIIIIX', (2+0j)), ('IIXIII', (2+0j)), ('IIIXII', (2+0j)), ('IIIIXI', (2+0j))]

... Trotterized Circuit, K=1, t=0.05
        ┌───────────┐ ░                                                      »
   q_0: ┤ U3(π,0,π) ├─░────────────────────────■─────────────────────────────»
        └───────────┘ ░                        │                             »
   q_1: ──────────────░────────────────────────┼─────────────────────────────»
        ┌───────────┐ ░                        │                             »
   q_2: ┤ U3(π,0,π) ├─░────────────────────────┼─────────────────────────────»
        └───────────┘ ░                        │                             »
   q_3: ──────────────░────────────────────────┼───────■──────────────────■──»
        ┌───────────┐ ░                        │     ┌─┴─┐   

### Compute Energy of the Hamiltonian - Unoptimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [5]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = False

# Create circuits from the Hamiltonian 
circuits = create_circuits_for_hamiltonian(num_qubits, ham_terms, use_commuting_groups)

for circuit in circuits:
    print(circuit)
    print(circuit[0])

N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)

    print("")
    print(f"... transpilation time = {round(time.time()-ts, 3)}")
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()

    # Compute the total energy for the Hamiltonian
    total_energy = calculate_expectation(num_qubits, results, circuits)

    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    
print("")


******** creating circuits from Hamiltonian:
  ... ZZIIII, (1+0j)
  ... ZIIIIZ, (1+0j)
  ... IZZIII, (1+0j)
  ... IIIIZZ, (1+0j)
  ... IIZZII, (1+0j)
  ... IIIZZI, (1+0j)
  ... XIIIII, (2+0j)
  ... IXIIII, (2+0j)
  ... IIIIIX, (2+0j)
  ... IIXIII, (2+0j)
  ... IIIXII, (2+0j)
  ... IIIIXI, (2+0j)

... constructed 12 circuits for this Hamiltonian.
(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000016BC1A7CFD0>, [('ZZIIII', (1+0j))])
         ░ ┌─┐               
   q_0: ─░─┤M├───────────────
         ░ └╥┘┌─┐            
   q_1: ─░──╫─┤M├────────────
         ░  ║ └╥┘┌─┐         
   q_2: ─░──╫──╫─┤M├─────────
         ░  ║  ║ └╥┘┌─┐      
   q_3: ─░──╫──╫──╫─┤M├──────
         ░  ║  ║  ║ └╥┘┌─┐   
   q_4: ─░──╫──╫──╫──╫─┤M├───
         ░  ║  ║  ║  ║ └╥┘┌─┐
   q_5: ─░──╫──╫──╫──╫──╫─┤M├
         ░  ║  ║  ║  ║  ║ └╥┘
meas: 6/════╩══╩══╩══╩══╩══╩═
            0  1  2  3  4  5 
(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000016BC1EEA290>, [('ZIIIIZ', (1+0j))])
 

### Compute Energy of the Hamiltonian - Optimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [6]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = True

# Create circuits from the Hamiltonian 
circuits = create_circuits_for_hamiltonian(num_qubits, ham_terms, use_commuting_groups)

for circuit in circuits:
    print(circuit)
    print(circuit[0])

N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)

    print("")
    print(f"... transpilation time = {round(time.time()-ts, 3)}")
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()

    # Compute the total energy for the Hamiltonian
    total_energy = calculate_expectation(num_qubits, results, circuits)

    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    
print("")


******** creating commuting groups for the Hamiltonian and circuits from the groups:
Group 1:
  ZZIIII: (1+0j)
  ZIIIIZ: (1+0j)
  IZZIII: (1+0j)
  IIIIZZ: (1+0j)
  IIZZII: (1+0j)
  IIIZZI: (1+0j)
Group 2:
  XIIIII: (2+0j)
  IXIIII: (2+0j)
  IIIIIX: (2+0j)
  IIXIII: (2+0j)
  IIIXII: (2+0j)
  IIIIXI: (2+0j)

... constructed 2 circuits for this Hamiltonian.
(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000016BC1F66410>, [('ZZIIII', (1+0j)), ('ZIIIIZ', (1+0j)), ('IZZIII', (1+0j)), ('IIIIZZ', (1+0j)), ('IIZZII', (1+0j)), ('IIIZZI', (1+0j))])
         ░ ┌─┐               
   q_0: ─░─┤M├───────────────
         ░ └╥┘┌─┐            
   q_1: ─░──╫─┤M├────────────
         ░  ║ └╥┘┌─┐         
   q_2: ─░──╫──╫─┤M├─────────
         ░  ║  ║ └╥┘┌─┐      
   q_3: ─░──╫──╫──╫─┤M├──────
         ░  ║  ║  ║ └╥┘┌─┐   
   q_4: ─░──╫──╫──╫──╫─┤M├───
         ░  ║  ║  ║  ║ └╥┘┌─┐
   q_5: ─░──╫──╫──╫──╫──╫─┤M├
         ░  ║  ║  ║  ║  ║ └╥┘
meas: 6/════╩══╩══╩══╩══╩══╩═
            0  1  2  3